# 20220411-degree tuning
- Phase behavior (per-trace, across simpoints)
    
- Collecting statistics (from `Pythia` directory):
    1. Create degree tuning sweep by running `./experiments/prefetcher_degree_sweep.py condor_setup`
    2. Evaluate degree tuning sweep by running `./experiments/prefetcher_degree_sweep.py eval`

In [12]:
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

from utils import utils

from importlib import reload
reload(utils)

<module 'utils.utils' from '/u/cmolder/GitHub/prefetching-zoo/utils/utils.py'>

In [8]:
data_path = '/u/cmolder/GitHub/Pythia/out/prefetcher_zoo_untuned.csv'
degree_path = '/u/cmolder/GitHub/Pythia/out/prefetcher_degree_sweep.csv'
prefetchers = ['sisb', 'bo', 'sms', 'spp']
metric = 'ipc_improvement'

df = utils.read_data_file(data_path)
df = df[df.prefetcher.isin(prefetchers)]
degree_df = utils.read_degree_sweep_file(degree_path)

In [3]:
df

,full_trace,trace,simpoint,prefetcher,degree,accuracy,coverage,mpki,mpki_reduction,dram_bw_epochs,dram_bw_reduction,ipc,ipc_improvement,baseline_prefetcher,path,baseline_path
6,astar_163B,astar,163B,bo,"(None,)",10.853989,3.434158,1.32554,33.222166,397657,4.480555,0.51985,4.042830,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
7,astar_163B,astar,163B,bo,"(1,)",14.005433,3.173354,1.32912,33.041814,397416,4.538445,0.52018,4.108876,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
8,astar_163B,astar,163B,bo,"(2,)",14.005433,3.173354,1.32912,33.041814,397416,4.538445,0.52018,4.108876,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
9,astar_163B,astar,163B,bo,"(3,)",13.389675,3.617740,1.32302,33.349118,397334,4.558142,0.52030,4.132893,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
10,astar_163B,astar,163B,bo,"(4,)",13.389675,3.617740,1.32302,33.349118,397334,4.558142,0.52030,4.132893,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3279,zeusmp_600B,zeusmp,600B,spp,"(4,)",99.726480,46.391020,1.69072,53.586841,171430,10.016167,1.24645,12.117042,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
3280,zeusmp_600B,zeusmp,600B,spp,"(5,)",99.734995,46.208384,1.69648,53.428719,171526,9.965776,1.24568,12.047781,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
3281,zeusmp_600B,zeusmp,600B,spp,"(6,)",99.734995,46.208384,1.69648,53.428719,171526,9.965776,1.24568,12.047781,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...
3282,zeusmp_600B,zeusmp,600B,spp,"(7,)",99.733105,46.483607,1.68780,53.667000,171344,10.061308,1.24713,12.178207,no,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...,/scratch/cluster/cmolder/prefetcher_zoo/2022_0...


In [4]:
degree_df

,Trace,sisb,sms,spp,bo
0,astar_313B,"(8,)","(5,)","(8,)","(1,)"
1,astar_23B,"(4,)","(6,)","(1,)","(2,)"
2,astar_163B,"(5,)","(8,)","(8,)","(3,)"
3,sphinx3_883B,"(8,)","(8,)","(1,)","(6,)"
4,sphinx3_2520B,"(8,)","(7,)","(7,)","(1,)"
...,...,...,...,...,...
57,tonto_422B,"(8,)","(8,)","(1,)","(4,)"
58,wrf_1212B,"(8,)","(3,)","(1,)","(7,)"
59,zeusmp_100B,"(2,)","(5,)","(1,)","(2,)"
60,zeusmp_300B,"(6,)","(5,)","(1,)","(7,)"


In [19]:
def get_tuning_improvement(data_df, degree_df, metric):
    prefs = degree_df.columns.to_list()
    prefs.remove('Trace')
    
    res = defaultdict(list)
    
    for tr in data_df.full_trace.unique():
        
        res['Trace'].append(tr)
        
        for pf in prefs:
            best_deg = degree_df[(degree_df.Trace == tr)][pf].item()
            baseline = data_df[(data_df.full_trace == tr) & (data_df.prefetcher == pf) & (data_df.degree == str((None,)))]
            comp     = data_df[(data_df.full_trace == tr) & (data_df.prefetcher == pf) & (data_df.degree == best_deg)]
            diff = comp[metric].item() - baseline[metric].item()
            res[pf].append(diff)
        
    df = pd.DataFrame(res)
            
    # Add mean
    mean = defaultdict(list)
    mean['Trace'] = 'amean' if metric in utils.amean_metrics else 'gmean'
    for pf in prefs:
        mean[pf].append(utils.mean(df[pf], metric))
    df = df.append(pd.DataFrame(mean))
    return df

pd.set_option('display.max_rows', None)
get_tuning_improvement(df, degree_df, metric)

,Trace,sisb,sms,spp,bo
0,astar_163B,0.026018,0.082057,0.038027,0.090063
1,astar_23B,-0.018864,0.240521,0.278249,-0.089606
2,astar_313B,12.030934,0.026364,0.087881,0.087881
3,bfs,8.500236,0.275008,-0.105558,-0.072224
4,bwaves_1609B,-0.089472,8.920896,0.166664,0.098244
5,bwaves_1861B,0.252991,8.784413,0.326780,0.014055
6,bwaves_98B,0.109161,0.020334,0.006421,0.142337
7,bzip2_183B,0.070523,0.148392,0.185123,0.000000
8,bzip2_259B,1.056875,0.608543,0.062017,0.304917
9,bzip2_281B,1.381795,-0.012768,0.228408,0.122007
